In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Gastroesophageal_reflux_disease_(GERD)"
cohort = "GSE28302"

# Input paths
in_trait_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)"
in_cohort_dir = "../../input/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE28302"

# Output paths
out_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/GSE28302.csv"
out_gene_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE28302.csv"
out_clinical_data_file = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/clinical_data/GSE28302.csv"
json_path = "../../output/preprocess/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide correct code for this step.

```python
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains genome-wide expression profiling
# using Illumina whole-genome Beadarray on RNA from esophageal biopsy tissues
is_gene_available = True

# 2. Data Availability and Conversion

# 2.1 Trait data - Barrett's esophagus related to GERD
trait_row = 0  # "tissue type" row

# Function to convert Barrett's esophagus data to binary values
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip()
    
    if "barrett" in value.lower() or "be" in value.lower():
        return 1  # Barrett's esophagus
    elif "normal" in value.lower() or "squamous" in value.lower():
        return 0  # Normal esophageal tissue (control)
    elif "adenocarcinoma" in value.lower() or "tumor" in value.lower():
        return None  # Exclude cancer samples as we're focusing on GERD/Barrett's
    return None

# 2.2 Age data
age_row = 4  # "subject age (years)" row

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# 2.3 Gender data
gender_row = 3  # "subject gender" row

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    if ":" in str(value):
        value = str(value).split(":", 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# 3. Save metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Create sample characteristics dictionary
    sample_char_dict = {
        0: ['tissue type: normal esophageal squamous', "tissue type: Barrett's esophagus (without dysplasia)", 'tissue type: esophageal adenocarcinoma tumor'],
        1: ['individual id: 53072', 'individual id: 53073', 'individual id: 54011', 'individual id: 52036', 'individual id: 53016', 'individual id: 53053', 'individual id: 53029', 'individual id: 53164', 'individual id: 52011', 'individual id: 53015', 'individual id: 54036', 'individual id: 54080', 'individual id: 52040', 'individual id: 54013', 'individual id: 53154', 'individual id: 52039', 'individual id: 54005', 'individual id: 54045', 'individual id: 54077', 'individual id: 53005', 'individual id: 53032', 'individual id: 53052', 'individual id: 54025', 'individual id: 53092', 'individual id: 53100', 'individual id: 53038', 'individual id: 53059', 'individual id: 53118', 'individual id: 53097', 'individual id: 53114'],
        2: ['histology review type (see paper for details): slide', 'histology review type (see paper for details): path info'],
        3: ['subject gender: female', 'subject gender: male'],
        4: ['subject age (years): 73', 'subject age (years): 55', 'subject age (years): 66', 'subject age (years): 21', 'subject age (years): 48', 'subject age (years): 41', 'subject age (years): 31', 'subject age (years): 80', 'subject age (years): 45', 'subject age (years): 75', 'subject age (years): 60', 'subject age (years): 72', 'subject age (years): 56', 'subject age (years): 47', 'subject age (years): 78', 'subject age (years): 65', 'subject age (years): 68', 'subject age (years): 43', 'subject age (years): 67', 'subject age (years): 69', 'subject age (years): 57', 'subject age (years): 77', 'subject age (years): 61', 'subject age (years): 79', 'subject age (years): 70', 'subject age (years): 62', 'subject age (years): 71', 'subject age (years): 63', 'subject age (years): 52', 'subject age (years): 74'],
        5: ['sample barcode: 1477791129_A', 'sample barcode: 1477791124_A', 'sample barcode: 1477791144_A', 'sample barcode: 1477791133_D', 'sample barcode: 1477791127_E', 'sample barcode: 1477791086_D', 'sample barcode: 1477791133_E', 'sample barcode: 1477791143_E', 'sample barcode: 1477791139_F', 'sample barcode: 1477791133_A', 'sample barcode: 1477791128_F', 'sample barcode: 1477791109_A', 'sample barcode: 1477791135_B', 'sample barcode: 1477791115_B', 'sample barcode: 1477791114_C', 'sample barcode: 1477791125_A', 'sample barcode: 1477791113_B', 'sample barcode: 1477791112_F', 'sample barcode: 1477791110_F', 'sample barcode: 1477791107_A', 'sample barcode: 1477791143_C', 'sample barcode: 1477791124_D', 'sample barcode: 1477791127_D', 'sample barcode: 1477791139_B', 'sample barcode: 1477791144_D', 'sample barcode: 1477791086_C', 'sample barcode: 1477791134_B', 'sample barcode: 1477791110_E', 'sample barcode: 1477791139_E', 'sample barcode: 1477791129_B'],
        6: [np.nan, 'matching cn sample id: GSM265790', 'matching cn sample id: GSM266075', 'matching cn sample id: GSM265786', 'matching cn sample id: GSM265500', 'matching cn sample id: GSM265789', 'matching cn sample id: GSM266703', 'matching cn sample id: GSM266074', 'matching cn sample id: GSM266706', 'matching cn sample id: GSM265808', 'sample barcode: 1477791107_E', 'matching cn sample id: GSM265787', 'matching cn sample id: GSM266708', 'matching cn sample id: GSM266660', 'matching cn sample id: GSM265809', 'matching cn sample id: GSM266705', 'matching cn sample id: GSM266707', 'matching cn sample id: GSM266119', 'matching cn sample id: GSM265501', 'matching cn sample id: GSM265791', 'matching cn sample id: GSM266715', 'matching cn sample id: GSM266659', 'matching cn sample


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
# Import necessary libraries
import pandas as pd
import os
import numpy as np
import json
from typing import Optional, Callable, Dict, Any, List, Union

# -------- 1. First, let's check what files are available in the input directory --------
print(f"Checking files in: {in_cohort_dir}")
available_files = os.listdir(in_cohort_dir)
print("Available files:", available_files)

# Look for appropriate files that might contain sample characteristics
potential_clinical_files = [f for f in available_files if 'clinical' in f.lower() or 'sample' in f.lower() or '.soft' in f.lower()]
print("Potential clinical files:", potential_clinical_files)

# If we find a SOFT file, let's use that as it contains complete information
soft_files = [f for f in available_files if f.endswith('.soft')]
if soft_files:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    print(f"Using SOFT file: {soft_file}")
    
    # Reading the SOFT file to extract sample characteristics
    with open(soft_file, 'r') as f:
        lines = f.readlines()
    
    # Extract sample characteristics
    sample_data = {}
    current_sample = None
    
    for line in lines:
        line = line.strip()
        if line.startswith("^SAMPLE"):
            parts = line.split(" = ")
            if len(parts) > 1:
                current_sample = parts[1]
                sample_data[current_sample] = {}
        elif line.startswith("!Sample_") and current_sample is not None:
            parts = line.split(" = ")
            if len(parts) > 1:
                key = parts[0].replace("!Sample_", "")
                value = parts[1]
                if key not in sample_data[current_sample]:
                    sample_data[current_sample][key] = value
                else:
                    if not isinstance(sample_data[current_sample][key], list):
                        sample_data[current_sample][key] = [sample_data[current_sample][key]]
                    sample_data[current_sample][key].append(value)
    
    # Convert to DataFrame
    all_keys = set()
    for sample_dict in sample_data.values():
        all_keys.update(sample_dict.keys())
    
    clinical_data = pd.DataFrame(index=list(sample_data.keys()), columns=list(all_keys))
    for sample, sample_dict in sample_data.items():
        for key, value in sample_dict.items():
            clinical_data.loc[sample, key] = value
    
    # Transpose to have characteristics as rows
    clinical_data = clinical_data.transpose()

else:
    # If no SOFT file, try to find a matrix file
    matrix_files = [f for f in available_files if 'matrix' in f.lower() or 'series_matrix' in f.lower()]
    if matrix_files:
        matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
        print(f"Using matrix file: {matrix_file}")
        
        # Read the file to extract sample characteristics
        with open(matrix_file, 'r') as f:
            lines = f.readlines()
        
        # Extract sample characteristics
        sample_data = {}
        sample_ids = []
        
        for line in lines:
            line = line.strip()
            if line.startswith("!Sample_"):
                parts = line.split("\t")
                key = parts[0].replace("!Sample_", "")
                values = parts[1:]
                
                if key == "geo_accession":
                    sample_ids = values
                    for sample_id in sample_ids:
                        sample_data[sample_id] = {}
                
                if sample_ids:
                    for i, sample_id in enumerate(sample_ids):
                        if i < len(values):
                            sample_data[sample_id][key] = values[i]
        
        # Convert to DataFrame
        all_keys = set()
        for sample_dict in sample_data.values():
            all_keys.update(sample_dict.keys())
        
        clinical_data = pd.DataFrame(index=list(sample_data.keys()), columns=list(all_keys))
        for sample, sample_dict in sample_data.items():
            for key, value in sample_dict.items():
                clinical_data.loc[sample, key] = value
        
        # Transpose to have characteristics as rows
        clinical_data = clinical_data.transpose()
    
    else:
        # If no suitable files found, create a dummy DataFrame and mark data as unavailable
        print("No suitable files found for clinical data.")
        clinical_data = pd.DataFrame()
        is_gene_available = False
        is_trait_available = False
        
        # Save metadata indicating data is not available
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )
        
        # Exit early
        print(f"Data not available for {cohort}. Metadata saved.")
        exit()

# Display the clinical data
print("Preview of clinical data:")
print(clinical_data.head())

# -------- 2. Check the unique values in each row to identify relevant information --------
unique_values = {}
for i in range(len(clinical_data.index)):
    row_name = clinical_data.index[i]
    values = clinical_data.iloc[i].unique()
    unique_values[i] = {
        "name": row_name,
        "values": values,
        "count": len(values)
    }
    print(f"Row {i} - {row_name}: {values}")

# -------- 3. Determine availability and conversion functions based on the data --------

# 3.1 Check if gene expression data is available
# Look for platform information that suggests gene expression
is_gene_available = True
platform_rows = [i for i, info in unique_values.items() if "platform" in str(info["name"]).lower()]
if platform_rows:
    platform_values = [str(v).lower() for v in unique_values[platform_rows[0]]["values"]]
    # If platform indicates miRNA or methylation, mark gene data as unavailable
    if any(("mirna" in v or "methylation" in v) for v in platform_values):
        is_gene_available = False

# 3.2 Identify the row indices for trait, age, and gender

# For GERD (Gastroesophageal reflux disease)
trait_row = None
for i, info in unique_values.items():
    row_name = str(info["name"]).lower()
    values = [str(v).lower() for v in info["values"]]
    
    # Look for rows that might contain GERD information
    if ("gerd" in row_name or "reflux" in row_name or "disease" in row_name or 
        "diagnosis" in row_name or "condition" in row_name or "group" in row_name):
        if any(("gerd" in v or "reflux" in v or "control" in v or "normal" in v or "disease" in v) for v in values):
            trait_row = i
            break

# If we couldn't find a direct trait row, check if we can infer from sample descriptions
if trait_row is None:
    for i, info in unique_values.items():
        row_name = str(info["name"]).lower()
        values = [str(v).lower() for v in info["values"]]
        
        if ("title" in row_name or "description" in row_name or "characteristic" in row_name):
            if any(("gerd" in v or "reflux" in v or "control" in v or "normal" in v) for v in values):
                trait_row = i
                break

# For age
age_row = None
for i, info in unique_values.items():
    row_name = str(


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Let's analyze the gene identifiers in the gene expression data

# The identifiers appear to start with 'GI_' followed by a number and a suffix like '-S', '-A', or '-I'
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# These look like GenBank or other database IDs that need to be mapped to standard gene symbols

# Looking at examples like:
# GI_10047089-S
# GI_10047091-S 
# These appear to be GenInfo Identifiers (GI numbers) which were used by NCBI

# Conclusion based on biomedical knowledge:
requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Get gene annotation data from the SOFT file (need to re-extract it)
gene_annotation = get_gene_annotation(soft_file)
print(f"Gene annotation shape: {gene_annotation.shape}")
print("Gene annotation columns:")
print(gene_annotation.columns.tolist())
print("Sample of gene annotation data:")
print(gene_annotation.head())

# 2. Create a gene mapping dataframe using the ID and GB_ACC columns
# ID column contains the same identifiers as in the gene expression data
# GB_ACC contains RefSeq accessions which we'll use for gene mapping
mapping_df = pd.DataFrame({
    'ID': gene_annotation['ID'],
    'Gene': gene_annotation['GB_ACC']
})
mapping_df = mapping_df.dropna(subset=['Gene'])  # Remove rows with missing gene information
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Define a custom function to extract gene identifiers from RefSeq accessions
# This is needed because the standard extract_human_gene_symbols function 
# wouldn't work well with RefSeq IDs
def extract_gene_from_refseq(refseq_id):
    """Extract a gene identifier from RefSeq accession without filtering"""
    if not isinstance(refseq_id, str):
        return []
    
    # For RefSeq accessions, return the accession without version number
    if refseq_id.startswith('NM_') or refseq_id.startswith('NR_') or refseq_id.startswith('XM_'):
        # Remove version number if present (e.g., NM_001234.2 -> NM_001234)
        base_id = refseq_id.split('.')[0]
        return [base_id]
    
    return []

# 4. Apply custom mapping to convert probe-level data to gene-level data
# First, modify the mapping DataFrame to use our custom extraction function
mapping_df['Gene'] = mapping_df['Gene'].apply(extract_gene_from_refseq)
mapping_df['num_genes'] = mapping_df['Gene'].apply(len)
mapping_df = mapping_df.explode('Gene')
mapping_df = mapping_df.dropna(subset=['Gene'])

if mapping_df.empty:
    print("WARNING: No valid mappings found between probes and genes")
    # Use probe IDs directly as fallback
    gene_data_mapped = gene_data.copy()
    gene_data_mapped.index = gene_data_mapped.index.map(lambda x: x.split('-')[0] if '-' in x else x)
else:
    # Set ID as index to allow joining with expression data
    mapping_df.set_index('ID', inplace=True)
    
    # Get expression columns (exclude ID column if it exists)
    expr_cols = [col for col in gene_data.columns if col != 'ID']
    
    # Join mapping with expression data
    merged_df = mapping_df.join(gene_data)
    
    # Distribute expression values based on number of genes per probe
    merged_df[expr_cols] = merged_df[expr_cols].div(merged_df['num_genes'].replace(0, 1), axis=0)
    
    # Sum expression values for each gene
    gene_data_mapped = merged_df.groupby('Gene')[expr_cols].sum()

# Use the mapped data
gene_data = gene_data_mapped

print(f"Gene data shape after mapping: {gene_data.shape}")
print("First few mapped genes:")
print(gene_data.index[:10].tolist())

# Save a sample of the gene expression data to observe its structure
print("Sample of gene expression data:")
print(gene_data.iloc[:5, :5] if not gene_data.empty else "No gene data available")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Load the gene expression data saved in step 6
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene data shape: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data (overwrite the previous file with normalized data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data created in step 2
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")

# If clinical_df doesn't have a proper index, fix it
if 'Unnamed: 0' in clinical_df.columns:
    clinical_df = clinical_df.set_index('Unnamed: 0')
elif not clinical_df.index.name:
    # Just in case the index needs to be set from data
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    clinical_df = clinical_features

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 3. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 4. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 5. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 6. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")